In [2]:
import gym
from gym.utils.play import play
import imageio
from skimage import color
from skimage import io
from skimage import img_as_ubyte
from skimage.util import crop
import numpy as np
import pickle
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def normalize_pixel(x):
    if x < 0.0:
        return 1.0
    elif x == 0.0:
        return 0.0
    else:
        return 0.5
normalize_pixel_v = np.vectorize(normalize_pixel)
class InputLine:
    def __init__(self, img, act) -> None:
        self.image = img
        self.action = act
first_enter = True
previous_frame = None
nb = 100
outfile = open('dataset_test.dat', 'ab')
cropping=((30, 10), (10, 10))
LARGEUR_IMAGE = 140
HAUTEUR_IMAGE = 170
def mycallback(obs_t, obs_tp1, action, rew, done, info):
    global previous_frame, first_enter, nb
    nb += 1
    if first_enter:
        previous_frame = crop(color.rgb2gray(obs_t),cropping ,copy=False)
        first_enter = False
    else:
        current_frame = crop(color.rgb2gray(obs_t), cropping,copy=False)
        combined_frame = normalize_pixel_v(previous_frame - current_frame)
        #print(current_frame)
        first_enter = True
        io.imsave(f"screenshots/skimage-greyscale-cropped-{nb}.png", img_as_ubyte(combined_frame))
        pickle.dump(InputLine(combined_frame, action),
                    outfile,
                    protocol=pickle.HIGHEST_PROTOCOL)
    #print("action = ", action, " reward = ", rew, "done = ", done)
env = gym.make('Pong-v4')
env.reset()
"""play(env,
     zoom=3,
     fps=12,
     callback=mycallback,
     keys_to_action={
         (ord('p'), ): 2,
         (ord('m'), ): 3
     })"""
env.close()
outfile.close()


x = []
y = []
x_train=[]
y_train=[]
x_test=[]
y_test=[]
with open('dataset.dat', 'rb') as handle:
    while True:
        try:
            obj = pickle.load(handle)
            x.append(obj.image)
            y.append(obj.action)
        except EOFError:
            break
        
    #print(arr)
y = tensorflow.keras.utils.to_categorical(y, 4)
x=np.reshape(x,(len(x),120*170))

for i in range(0,len(x)):
    if i%8==0 or i%9==0:
        x_test.append(x[i])
        y_test.append(y[i])
    else:
        x_train.append(x[i])
        y_train.append(y[i])

        
x_train=np.array(x_train)
y_train=np.array(y_train)
print("X_train = ",x_train.shape)
print("y_train = ",(y_train).shape)

x_test=np.array(x_test)
y_test=np.array(y_test)
print("X_test = ",x_test.shape)
print("y_test = ",(y_test).shape)


model = tensorflow.keras.Sequential()
model.add(layers.Dense(16, input_dim=LARGEUR_IMAGE*HAUTEUR_IMAGE, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, validation_split=0.33)


score = model.evaluate(x_train,y_train)
score_test=model.evaluate(x_test,y_test)

env = gym.make('Pong-v4')
env.reset()
first_enter=True
previous_frame = None
cropping=((30, 10), (10, 10))
agent=model
move=None
while True:
    env.render()
    if move!=None:
        if move!=0:
            print(move)
        obs,rew,d,inf=env.step(move)
    else:
        obs,rew,d,inf=env.step(env.action_space.sample())
    if d:
        break
    if first_enter:
        previous_frame = crop(color.rgb2gray(obs), cropping,copy=False)
        first_enter=False
    else:
        current_frame = crop(color.rgb2gray(obs), cropping,copy=False)
        combined_frame = normalize_pixel_v(previous_frame - current_frame)
        combined_frame=np.reshape(combined_frame,(20400))
        move=agent.predict(np.array([combined_frame]))[0]
        move=(np.argmax(move,axis=0))
        first_enter=True
env.close()


X_train =  (235, 20400)
y_train =  (235, 4)
X_test =  (67, 20400)
y_test =  (67, 4)
Epoch 1/100


ValueError: in user code:

    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
       